In [1]:
import ee 
ee.Initialize()

In [2]:
import geemap
Map = geemap.Map()
# Map

In [3]:

from typing import Dict
import ee
ee.Initialize()

from easydict import EasyDict as edict
from prettyprinter import pprint

import logging
logger = logging.getLogger(__name__)

# from eo_class.modisPoly import MODIS_POLY
import yaml
import numpy as np
import json

""" CFG """
cfg = edict({
    "JSON": "G:\PyProjects\wildfire-s1s2-dataset\wildfire_events\MTBS_AK_2017_2019_events_ROI.json",
    "period": 'fire_period', # "season"
    "season": [-1, 2],
})
pprint(cfg)

""" Fire Event """
from eo_class.fireEvent import load_json

EVENT_SET = load_json(cfg.JSON)
event = EVENT_SET['ak6569814836520190622']
# pprint(event)

queryEvent = edict(event.copy())
queryEvent['start_date'] = event['Ig_Date']
queryEvent['end_date'] = event['modisEndDate']
queryEvent['year'] = ee.Number(event['YEAR']).format().getInfo()

# pprint(queryEvent)

queryEvent['roi'] = ee.Geometry.Polygon(event['roi']).bounds()


# pprint(queryEvent)

# fire period
if 'fire_period' in cfg.period:
    period_start = ee.Date(queryEvent['start_date']).advance(-1, 'month')
    period_end = ee.Date(queryEvent['start_date']).advance(2, 'month')

# season period
if 'season' in cfg.period:
    period_start = ee.Date(ee.Number(queryEvent['year']).format()).advance(cfg.season[0], 'month')
    period_end = ee.Date(ee.Number(queryEvent['year']).format()).advance(cfg.season[-1], 'month')

print(period_start.format().getInfo(), period_end.format().getInfo())



easydict.EasyDict({
    'JSON':
        'G:\\PyProjects\\wildfire-s1s2-dataset\\wildfire_events\\'
        'MTBS_AK_2017_2019_events_ROI.json',
    'period': 'fire_period',
    'season': [-1, 2]
})
2019-05-22T00:00:00 2019-08-22T00:00:00


In [4]:
def get_pntsFilter(roi, buffer_size=-2e3):
    ## ==> Point Filters <==
    pnt_roi = roi.buffer(buffer_size, ee.ErrorMargin(1)).bounds()
    coordList = ee.List(pnt_roi.coordinates().get(0))
    btmLeft = ee.Geometry.Point(coordList.get(0))
    btmRight = ee.Geometry.Point(coordList.get(1))
    topRight = ee.Geometry.Point(coordList.get(2))
    topLeft = ee.Geometry.Point(coordList.get(3))
    pntsFC = ee.FeatureCollection([btmLeft, btmRight, topRight, topLeft]).style(color="red")

    # if len(pntsList) > 0:
    pntsFilter = ee.Filter.And(
        ee.Filter.geometry(btmLeft),
        ee.Filter.geometry(btmRight),
        ee.Filter.geometry(topRight),
        ee.Filter.geometry(topLeft))
    
    return pntsFilter, pntsFC

pntsFilter, pntsFC = get_pntsFilter(queryEvent.roi)

# Query S2 MSI Data

In [5]:
# """ Query S2 MSI Data """

# S2_Dict = {
#     'SR': "COPERNICUS/S2_SR",
#     'TOA': "COPERNICUS/S2"
# }

# L8_Dict = {
#     'SR': "LANDSAT/LC08/C01/T1_SR",
#     'TOA': "LANDSAT/LC08/C01/T1_TOA"
# }


# MSI = ee.ImageCollection(S2_Dict['TOA']).filterBounds(queryEvent['roi'])
# cloudFilter = ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 30)

# MSI_pre = MSI.filterDate(period_start.advance(-1, 'year'), period_end.advance(-1, "year")).filter(cloudFilter).median()
# MSI_post = MSI.filterDate(period_start.advance(1, 'year'), period_end.advance(1, "year")).filter(cloudFilter).median()

# Query S1 SAR Data

In [6]:
""" Query S1 SAR Data """

# Sentinel-1
def set_group_index_4_S1(img, groupLevel=13, labelShowLevel=13):
        orbitKey = (ee.String(img.get("orbitProperties_pass")).replace('DESCENDING', 'DSC').replace('ASCENDING', 'ASC')
                    .cat(ee.Number(img.get("relativeOrbitNumber_start")).int().format()))
        Date = (img.date().format().slice(0, labelShowLevel)
                    .replace('-', '').replace('-', '').replace(':', '').replace(':', ''))
        Name = (Date).cat('_').cat(orbitKey)

        groupIndex = img.date().format().slice(0, groupLevel)  # 2017 - 07 - 23T14:11:22(len: 19)
        return img.setMulti({
            'GROUP_INDEX': groupIndex,
            'IMG_LABEL': Name,
            'Orbit_Key': orbitKey
        })

def unionGeomFun(img, first):
    rightGeo = ee.Geometry(img.geometry())
    return ee.Geometry(first).union(rightGeo)

def toNatural(img):
        return ee.Image(10.0).pow(img.divide(10.0))

def toDB(img):
    return ee.Image(img).log10().multiply(10.0)

def add_RFDI(img):
    RFDI = toNatural(img.select(['VV','VH'])).normalizedDifference(['VV', 'VH']).select('nd').rename('ND')
    RFDI_dB = toDB(RFDI).multiply(10)

    return img.addBands(RFDI_dB).copyProperties(img, img.propertyNames())

def add_CR(img):
    return img.addBands(img.expression("b('VH')-b('VV')").rename('CR'))


# start to query S1 data
S1_flt = ee.ImageCollection(ee.ImageCollection("COPERNICUS/S1_GRD")
                        .filterBounds(queryEvent.roi)
                        .filterMetadata('instrumentMode', "equals", 'IW')
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                    ).select(['VH', 'VV'])


S1_pre = (S1_flt.filterDate(period_start.advance(-1, 'year'), period_end.advance(-1, "year"))
                .map(add_RFDI)
                .map(set_group_index_4_S1)
        )

S1_post = (S1_flt.filterDate(period_start.advance(1, 'year'), period_end.advance(1, "year"))
                .map(add_RFDI)
                .map(set_group_index_4_S1)
        )

# SAR orbits
pre_orbits = S1_pre.aggregate_array("Orbit_Key").distinct().getInfo()
post_orbits = S1_post.aggregate_array("Orbit_Key").distinct().getInfo()
common_orbits = list(set(pre_orbits) & set(post_orbits))

# Compute average image for each orbit
S1 = {}
for orbit in common_orbits:
# for orbit in ['DSC160']:
    orbImgCol_pre = S1_pre.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)
    orbImgCol_post = S1_post.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)

    orb_images = orbImgCol_pre.filter(ee.Filter.eq("IMG_LABEL", orbImgCol_pre.first().get("IMG_LABEL")))
    firstImgGeom = orb_images.first().geometry()
    orb_geom = ee.Geometry(orb_images.iterate(unionGeomFun, firstImgGeom))

    if orb_geom.contains(queryEvent.roi, ee.ErrorMargin(1)).getInfo():
        S1[f"{orbit}_pre"] = orbImgCol_pre.mean()
        S1[f"{orbit}_post"] = orbImgCol_post.mean()

        # Map.addLayer(orb_geom, {}, f"{orbit}_geom")
        # Map.addLayer(S1[f"{orbit}_pre"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_pre")
        # Map.addLayer(S1[f"{orbit}_post"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_post")
        

In [7]:
S1

{'DSC131_pre': <ee.image.Image at 0x241ffade648>,
 'DSC131_post': <ee.image.Image at 0x241ffae05c8>,
 'ASC94_pre': <ee.image.Image at 0x241ffab7208>,
 'ASC94_post': <ee.image.Image at 0x241ffad2308>}

In [8]:
Map.centerObject(queryEvent.roi, 6)

def unionGeomFun(img, first):
    rightGeo = ee.Geometry(img.geometry())
    return ee.Geometry(first).union(rightGeo)

S1 = {}
for orbit in common_orbits:
# for orbit in ['DSC160']:
    orbImgCol_pre = S1_pre.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)
    orbImgCol_post = S1_post.filter(ee.Filter.eq("Orbit_Key", orbit))#.sort("GROUP_INDEX", False)

    orb_images = orbImgCol_pre.filter(ee.Filter.eq("IMG_LABEL", orbImgCol_pre.first().get("IMG_LABEL")))
    firstImgGeom = orb_images.first().geometry()
    orb_geom = ee.Geometry(orb_images.iterate(unionGeomFun, firstImgGeom))

    if orb_geom.contains(queryEvent.roi, ee.ErrorMargin(1)).getInfo():
        S1[f"{orbit}_pre"] = orbImgCol_pre.mean()
        S1[f"{orbit}_post"] = orbImgCol_post.mean()

        # Map.addLayer(orb_geom, {}, f"{orbit}_geom")
        Map.addLayer(S1[f"{orbit}_pre"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_pre")
        Map.addLayer(S1[f"{orbit}_post"], {"bands": ['VH'], "min":-20, "max": -10}, f"{orbit}_post")
        
Map.addLayer(queryEvent['roi'], {}, 'roi', True, 0.3)

In [9]:
S1 

{'DSC131_pre': <ee.image.Image at 0x241ffac5348>,
 'DSC131_post': <ee.image.Image at 0x241ffab93c8>,
 'ASC94_pre': <ee.image.Image at 0x241ff762b08>,
 'ASC94_post': <ee.image.Image at 0x241ffac5b08>}

In [10]:
checkOrb = "ASC94"
S1_MT = ee.Image.cat([S1[f'{checkOrb}_pre'], S1[f'{checkOrb}_post']])
Map.addLayer(S1_MT, {'bands':['VH', 'VH_1', 'VH_1'], 'min': -20, 'max': -10}, f'S1_MT: {checkOrb}')

# ALOS

In [7]:
def toDB(img):
    return ee.Image(img).log10().multiply(10.0)
    
def add_ALOS_RFDI(img):
    RFDI = img.select(['HH','HV']).normalizedDifference(['HH','HV']).select('nd').rename('ND')
    RFDI_dB = toDB(RFDI).multiply(3)
    return img.addBands(RFDI_dB)

def to_ALOS_dB(img):
    return img.addBands(img.select(['HH', 'HV']).pow(ee.Image(2)).log10().multiply(10).subtract(83), ['HH', 'HV'], True)

def add_ALOS_CR(img):
    return img.addBands(img.expression("b('HV')-b('HH')").rename('CR'))


# ALOS
PALSAR = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR")
PALSAR_dB = PALSAR.map(add_ALOS_RFDI).map(to_ALOS_dB)#.map(add_ALOS_CR)

before_year = ee.Number.parse(queryEvent.year).subtract(1).format().getInfo()
after_year = ee.Number.parse(queryEvent.year).add(1).format().getInfo()

ALOS_pre = PALSAR_dB.filter(ee.Filter.eq("system:index", before_year)).first()
ALOS_post = PALSAR_dB.filter(ee.Filter.eq("system:index", after_year)).first()

ALOS = edict()
ALOS['pre'] = ALOS_pre.select(["ND", "HV", "HH"])
ALOS['post'] = ALOS_pre.select(["ND", "HV", "HH"])

In [8]:
Map.addLayer(ALOS['post'].clip(queryEvent.roi), {"bands": ["ND", "HV", "HH"], "min":-20, "max": -10}, f"alos_post")

In [28]:
after_year

'2020'

In [9]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# Mask

In [14]:
modis.bandNames().getInfo()

['BurnDate', 'Uncertainty', 'QA', 'FirstDay', 'LastDay']

In [15]:
firecci.bandNames().getInfo()

['BurnDate', 'ConfidenceLevel', 'LandCover', 'ObservedFlag']

In [17]:
WHERE = "AK"  # or "US"
mask_dict = edict()

# MODIS
MODIS = ee.ImageCollection("MODIS/006/MCD64A1")
modis = MODIS.filterDate(ee.Date(queryEvent.year), ee.Date(queryEvent.year).advance(1, 'year')).mosaic()
mask_dict['modis'] = modis.select("BurnDate")

# FireCCI51
FireCCI51 = ee.ImageCollection("ESA/CCI/FireCCI/5_1")
firecci = FireCCI51.filterDate(ee.Date(queryEvent.year), ee.Date(queryEvent.year).advance(1, 'year')).mosaic()
mask_dict['firecci'] = firecci.select("BurnDate")

# Water
landCover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2017")
landCover = ee.Image(landCover.select("discrete_classification").rename('CGLS').setMulti({'IMG_LABEL': 'CGLS'}))
water = (landCover.neq(80).And(landCover.neq(200))).rename('water')
mask_dict['water'] = water

# Polygon
if WHERE in ['AK', 'US']:
    print(WHERE)

    # USA
    US_bef_2018 = ee.FeatureCollection("users/omegazhangpzh/C_US_Fire_Perimeters/US_HIST_FIRE_PERIMTRS_2000_2018_DD83")
    US_2019 = ee.FeatureCollection("users/omegazhangpzh/C_US_Fire_Perimeters/US_HIST_FIRE_PERIM_2019_dd83")
    poly = (US_bef_2018.filterBounds(queryEvent.roi).filter(ee.Filter.gte("fireyear", 2017))
                .merge(US_2019.filterBounds(queryEvent.roi))
                .filter(ee.Filter.gte("gisacres", 5000)) # burned area
            ).union(ee.ErrorMargin(30))
    polyImg = poly.style(color='white', fillColor='white', width=0).select('vis-red').gt(0).rename('poly')
    mask_dict['poly'] = polyImg
                
    # USA MTBS
    mtbs = ee.Image.loadGeoTIFF(f"gs://eo4wildfire/US_BurnSeverityRaster/mtbs_{WHERE}_{queryEvent.year}.tif")
    mask_dict['mtbs'] = mtbs

if WHERE in ['CA']:
    pass

if WHERE in ['EU']:
    pass


AK


In [21]:
mask_dict

{'modis': <ee.image.Image at 0x241824ea608>,
 'firecci': <ee.image.Image at 0x2418239f488>,
 'water': <ee.image.Image at 0x241824ce8c8>,
 'poly': <ee.image.Image at 0x24181ca0e88>,
 'mtbs': <ee.image.Image at 0x2418248d288>}

In [18]:
Map.addLayer(mtbs.clip(queryEvent.roi), {"min":0, "max": 6}, 'mtbs')
Map.addLayer(modis.clip(queryEvent.roi), {}, 'modis')
Map.addLayer(firecci.clip(queryEvent.roi), {}, 'firecci')
Map.addLayer(polyImg, {min:0, max: 1}, 'poly')
Map.addLayer(water, {min:0, max: 1}, 'water')


In [19]:
Map

Map(bottom=4497.0, center=[65.61219222118567, -148.14545638310784], controls=(WidgetControl(options=['position…

In [9]:
orbit

'DSC160'

In [26]:
ee.ImageCollection(s1_pre_tmp).filter(pntsFilter).size().getInfo()

1

In [13]:
orbImgCol_post.mosaic().geometry().getInfo()

{'type': 'Polygon',
 'coordinates': [[[-180, -90],
   [180, -90],
   [180, 90],
   [-180, 90],
   [-180, -90]]]}

In [4]:
Map.centerObject(queryEvent.roi, 10)
Map.addLayer(MSI_pre, {"bands": ['B12', 'B8', 'B4'], "min":0, "max": 3000}, 'MSI_pre')
Map.addLayer(MSI_post, {"bands": ['B12', 'B8', 'B4'], "min":0, "max": 3000}, 'MSI_post')

Map.addLayer(queryEvent['roi'], {}, 'roi')

In [30]:
Map.addLayer(pntsFC, {}, 'pntsFC')

In [32]:
Map.addLayer(S1_pre.filter(ee.Filter.eq("Orbit_Key", orbit)).first().geometry(), {}, 'geom')